In [1]:
import wget
import pandas as pd

In [2]:
# Download and read the `spacex_launch_geo.csv`
spacex_csv_file = wget.download('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv')
spacex_df=pd.read_csv(spacex_csv_file)

In [3]:
# Import required libraries
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

/tmp/ipykernel_8248/1399112611.py:4: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
/tmp/ipykernel_8248/1399112611.py:5: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc


In [4]:
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

In [5]:
# Create a dash application
app = dash.Dash(__name__)

In [6]:
# Create an app layout
app.layout = html.Div(children=[
    html.H1('SpaceX Records Dashboard',style={'textAlign': 'center', 'color': '#503D36','font-size': 40}),
    # TASK 1: Add a dropdown list to enable Launch Site selection.The default select value is for ALL sites
    dcc.Dropdown(id='launch-site',
                 options=[
                     {'label': 'All Sites', 'value': 'ALL'},
                     {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                     {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'},
                     {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                     {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'}],
                 value='ALL',placeholder="place holder here", searchable=True),
# TASK 2: Add a pie chart to show the total successful launches count for all sites
# If a specific launch site was selected, show the Success vs. Failed counts for the site
    dcc.Graph(id='success-pie-chart'),
# TASK 3: Add a slider to select payload range
    html.P('Payload Mass (kg):'),
    dcc.RangeSlider(id = 'payload-slider',
                   min=0, max=10000, step=1000,
                   marks={0:'0', 100:'100'},
                   value = [min_payload, max_payload]),
# TASK 4: Add a scatter chart to show the correlation between payload and launch success
    html.Div(dcc.Graph(id='success-payload-scatter-chart')),
])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
# Function decorator to specify function input and output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='launch-site', component_property='value'))
def get_piechart(site):
    df = spacex_df[['Launch Site', 'class']]
    if site == 'ALL':
        df = pd.DataFrame(df.groupby('Launch Site').sum())
        df.reset_index(inplace=True)
        fig = px.pie(df, values='class', names='Launch Site', title='Total success launches by site')
        return fig
    else:
        df = df[df['Launch Site']== site]
        df = pd.DataFrame(df['class'].value_counts())
        df.reset_index(inplace=True)
        fig1 = px.pie(df, values='count', names='class', title='Total success launches for site')
        return fig1

# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(Output(component_id = 'success-payload-scatter-chart', component_property='figure'),
              Input(component_id='payload-slider', component_property='value'), 
              Input(component_id='launch-site', component_property='value')
             )
#def get_ouput(value):
 #   return 'you have selected {}'.format(value)
def get_scatterchart(value, site):
    df = spacex_df[['Launch Site','Booster Version', 'Payload Mass (kg)', 'class']]
    if site == 'ALL':
        fig=px.scatter(df,x='Payload Mass (kg)',y='class', color='Booster Version',title='Correlation between Payload & Success')
        fig.update_xaxes(range=value)
        return fig
    else:
        df = df[df['Launch Site']==site]
        fig = px.scatter(df, x='Payload Mass (kg)', y='class', color='Booster Version', title='Correlation between Payload & Success')
        fig.update_xaxes(range=value)
        return fig

# Run the app
if __name__ == '__main__':
    app.run_server()


In [7]:
spacex_df.head()

,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Landing Outcome,class,Lat,Long
0,1,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Failure (parachute),0,28.562302,-80.577356
1,2,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel o...",0.0,LEO (ISS),NASA (COTS) NRO,Failure (parachute),0,28.562302,-80.577356
2,3,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2+,525.0,LEO (ISS),NASA (COTS),No attempt,0,28.562302,-80.577356
3,4,2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356
4,5,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356


In [8]:
def get_scatterchart(site, payload_mass):
    df = spacex_df[['Launch Site','Booster Version', 'Payload Mass (kg)', 'class']]
    if site == 'ALL':
        df = df[['Payload Mass (kg)', 'class', 'Booster Version']]
        fig=px.scatter(df,x='Payload Mass (kg)',y='class', color='Booster Version')
        return fig
    else:
        df = df[df['Launch Site']==site]
        fig = px.scatter(df, x='Payload Mass (kg)', y='class', color='Booster Version')
        return fig

In [9]:
df = spacex_df[['Launch Site','Booster Version', 'Payload Mass (kg)', 'class']]
fig=px.scatter(df,x='Payload Mass (kg)',y='class', color='Booster Version')
fig.show()

ValueError: Invalid value

In [ ]:
df = spacex_df[['Launch Site','Booster Version', 'Payload Mass (kg)', 'class']]
df = df[df['Launch Site']=='CCAFS LC-40']
fig = px.scatter(df, x='Payload Mass (kg)', y='class', color='Booster Version')
fig.show()

In [ ]:
spacex_df.columns

In [ ]:
def get_scatterchart(site, slider):
    df = spacex_df[['Launch Site','Booster Version', 'Payload Mass (kg)', 'class']]
    if site == 'ALL':
        fig=px.scatter(df,x='Payload Mass (kg)',y='class', color='Booster Version')
        fig.update_xaxes(range=slider)
        return fig
    else:
        df = df[df['Launch Site']==site]
        fig = px.scatter(df, x='Payload Mass (kg)', y='class', color='Booster Version')
        fig.update_xaxes(range=slider)
        return fig

In [ ]:
slider = [500,5000]
get_scatterchart('ALL', slider)

In [ ]:
get_scatterchart('CCAFS LC-40', 200)